In [ ]:
from game import Game
import time
import os
import re

## Load maps

In [ ]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [ ]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

In [ ]:
game.get_box_locations()

In [ ]:
game.get_goal_locations()

In [ ]:
game.get_player_position()

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [ ]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

In [ ]:
game.apply_move('U')
game.display_map()

In [ ]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

## Solvers

### BFS

In [ ]:
from collections import deque
# TODO: Must return moves (if there is no solution return None), number of visited states
# def solver_bfs(game_map):
#     game = Game(game_map)
#     return None, 0
def solver_bfs(game_map):    
    game = Game(game_map)
    
    initial_player_pos = game.get_player_position()
    initial_boxes = game.get_box_locations()
    goals = sorted(game.get_goal_locations())
    
    if sorted(initial_boxes) == goals:
        return [], 1
    
    queue = deque([(initial_player_pos, tuple(sorted(initial_boxes)), [])])
    visited = {(initial_player_pos, tuple(sorted(initial_boxes)))}
    visited_count = 1
    
    test_game = Game(game_map)
    
    directions = ['U', 'D', 'L', 'R']
    
    while queue:
        player_pos, boxes, moves = queue.popleft()
        
        boxes_list = list(boxes)
        
        for move in directions:
            test_game.set_player_position(player_pos)
            test_game.set_box_positions(boxes_list)
            
            if test_game.apply_move(move):
                new_player_pos = test_game.get_player_position()
                new_boxes = test_game.get_box_locations()
                
                if sorted(new_boxes) == goals:
                    return moves + [move], visited_count + 1
                
                new_boxes_tuple = tuple(sorted(new_boxes))
                new_state = (new_player_pos, new_boxes_tuple)
                
                if new_state not in visited:
                    visited.add(new_state)
                    visited_count += 1
                    queue.append((new_player_pos, new_boxes_tuple, moves + [move]))
    
    return None, visited_count

### DFS

In [ ]:
import time
# TODO: Must return moves, number of visited states
# def solver_dfs(game_map):
#     game = Game(game_map)
#     return None, 0
def solver_dfs(game_map, max_depth=1000):
    game = Game(game_map)
    initial_player_pos = game.get_player_position()
    initial_boxes = game.get_box_locations()
    goals = sorted(game.get_goal_locations())

    if sorted(initial_boxes) == goals:
        return [], 1

    stack = [(initial_player_pos, tuple(sorted(initial_boxes)), [])]
    visited = {(initial_player_pos, tuple(sorted(initial_boxes)))}
    visited_count = 1

    test_game = Game(game_map)

    directions = ['U', 'D', 'L', 'R']

    while stack:
        player_pos, boxes, moves = stack.pop() 
        
        if len(moves) >= max_depth:
            continue
            
        boxes_list = list(boxes)
        
        #reverse order so U is pushed last and popped first
        for move in reversed(directions):
            test_game.set_player_position(player_pos)
            test_game.set_box_positions(boxes_list)
            
            if test_game.apply_move(move):
                new_player_pos = test_game.get_player_position()
                new_boxes = test_game.get_box_locations()
                
                if sorted(new_boxes) == goals:
                    return moves + [move], visited_count + 1
                
                new_boxes_tuple = tuple(sorted(new_boxes))
                new_state = (new_player_pos, new_boxes_tuple)
                
                if new_state not in visited:
                    visited.add(new_state)
                    visited_count += 1
                    stack.append((new_player_pos, new_boxes_tuple, moves + [move]))
    return None, visited_count

### IDS

In [ ]:
# TODO: Must return moves, number of visited states
# def solver_ids(game_map):
#     game = Game(game_map)
#     return None, 0
def solver_ids(game_map, max_depth=100):
    total_visited_count = 0

    for depth_limit in range(1, max_depth + 1):
        result, visited_count = depth_limited_search(game_map, depth_limit)
        total_visited_count += visited_count
        
        if result is not None:
            return result, total_visited_count
    
    return None, total_visited_count


def depth_limited_search(game_map, depth_limit):
    game = Game(game_map)
    
    initial_player_pos = game.get_player_position()
    initial_boxes = game.get_box_locations()
    goals = sorted(game.get_goal_locations())
    
    if sorted(initial_boxes) == goals:
        return [], 1
    
    stack = [(initial_player_pos, tuple(sorted(initial_boxes)), [], 0)]
    visited = {(initial_player_pos, tuple(sorted(initial_boxes)), 0)}  # Include depth in visited state
    visited_count = 1
    
    test_game = Game(game_map)
    
    directions = ['U', 'D', 'L', 'R']
    
    while stack:
        player_pos, boxes, moves, depth = stack.pop()
        
        if depth >= depth_limit:
            continue
            
        boxes_list = list(boxes)
        
        for move in reversed(directions):
            test_game.set_player_position(player_pos)
            test_game.set_box_positions(boxes_list)
            
            if test_game.apply_move(move):
                new_player_pos = test_game.get_player_position()
                new_boxes = test_game.get_box_locations()
                new_depth = depth + 1
                
                if sorted(new_boxes) == goals:
                    return moves + [move], visited_count + 1
                
                new_boxes_tuple = tuple(sorted(new_boxes))
                new_state = (new_player_pos, new_boxes_tuple, new_depth)
                if new_state not in visited:
                    visited.add(new_state)
                    visited_count += 1
                    stack.append((new_player_pos, new_boxes_tuple, moves + [move], new_depth))
    return None, visited_count

### A*

In [ ]:
import heapq
# TODO
# def heuristic(game_map):
#     return 0
def heuristic(game):
    # boxes = game.get_box_locations()
    # goals = game.get_goal_locations()
    # player_pos = game.get_player_position()
    
    # if sorted(boxes) == sorted(goals):
    #     return 0
    
    # total_distance = 0
    
    # for box in boxes:
    #     min_dist = min(manhattan_distance(box, goal) for goal in goals)
    #     total_distance += min_dist
    
    # min_player_dist = min(manhattan_distance(player_pos, box) for box in boxes)
    # total_distance += min_player_dist * 0.5
    
    # return total_distance
    boxes = game.get_box_locations()
    goals = game.get_goal_locations()
    
    if sorted(boxes) == sorted(goals):
        return 0
    
    boxes_sorted = sorted(boxes)
    goals_sorted = sorted(goals)
    
    total_distance = 0
    for i in range(len(boxes_sorted)):
        total_distance += manhattan_distance(boxes_sorted[i], goals_sorted[i])
    
    return total_distance

def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

# TODO: Must return moves, number of visited states
# def solver_astar(game_map, heuristic_func=heuristic, weight=1):
#     game = Game(game_map)
#     return None, 0
def solver_astar(game_map, heuristic_func=heuristic, weight=1):        
    game = Game(game_map)
    
    initial_player_pos = game.get_player_position()
    initial_boxes = game.get_box_locations()
    goals = sorted(game.get_goal_locations())
    
    if sorted(initial_boxes) == goals:
        return [], 1
    
    open_set = []
    game.set_player_position(initial_player_pos)
    game.set_box_positions(initial_boxes)
    initial_h_score = heuristic_func(game)
    
    import itertools
    counter = itertools.count()
    
    heapq.heappush(open_set, (initial_h_score, next(counter), 0, initial_player_pos, 
                             tuple(sorted(initial_boxes)), []))
    
    visited = {(initial_player_pos, tuple(sorted(initial_boxes))): 0}
    visited_count = 1
    
    test_game = Game(game_map)
    
    directions = ['U', 'D', 'L', 'R']
    
    while open_set:
        _, _, move_count, player_pos, boxes, moves = heapq.heappop(open_set)
        
        boxes_list = list(boxes)
        
        if sorted(boxes_list) == goals:
            return moves, visited_count
        
        for move in directions:
            test_game.set_player_position(player_pos)
            test_game.set_box_positions(boxes_list)
            
            if test_game.apply_move(move):
                new_player_pos = test_game.get_player_position()
                new_boxes = test_game.get_box_locations()
                new_move_count = move_count + 1
                
                new_boxes_tuple = tuple(sorted(new_boxes))
                new_state = (new_player_pos, new_boxes_tuple)
                
                if new_state not in visited or new_move_count < visited[new_state]:
                    visited[new_state] = new_move_count
                    visited_count += 1
                    
                    test_game.set_player_position(new_player_pos)
                    test_game.set_box_positions(new_boxes)
                    h_score = heuristic_func(test_game)
                    
                    f_score = new_move_count + (weight * h_score)
                    heapq.heappush(open_set, (f_score, next(counter), new_move_count, 
                                             new_player_pos, new_boxes_tuple, moves + [move]))
    
    return None, visited_count

## Solve

In [ ]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [ ]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [ ]:
solve(1, "BFS") # Solve map 1 using BFS

In [ ]:
import time
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            if (method == "BFS" and map == 9 ):
                continue
            if (method == "DFS" and map >= 7 ):
                continue
            if (method == "IDS" and map != 8 and map >=7):
                continue
            if( method == "A*" and map == 9 ):
                continue
            else : 
                solve(map, method)
def solve_Astarweighted():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):                   
        
        file_name = "map" + str(map) + ".txt"
        with open('./assets/maps/' + file_name) as f:
            game_map = f.read()
        
        weights = [1.2, 1.5, 2.0]
        for weight in weights:
            start_time = time.time()
            solution, visited_count = solver_astar(game_map, weight=weight)
            end_time = time.time()
            
            print(f"weighted_A* (w={weight}) took {round(end_time - start_time, 2)} seconds on map {map} and visited {visited_count} states.")
            if solution is None:
                print("No Solution Found!")
            else:
                print(f"{len(solution)} moves were used: {solution}")


In [ ]:
solve_all() # Solve all maps using all methods
solve_Astarweighted()